# INTERCEPT PARSER

In [ ]:
import argparse
from unittest.mock import patch
from openbb_terminal.economy import economy_controller

ec = economy_controller.EconomyController(queue=[])

parser_list = []


def mock_func(parser: argparse.ArgumentParser, *args, **kwargs):
    parser_list.append(parser)
    return None


with patch.object(ec, "parse_known_args_and_warn", new=mock_func) as mock_method:
    ns = ec.call_overview(other_args=["--help"])

parser_list

# GET ALL CONTROLLER CLASSES

In [ ]:
from pathlib import Path
from inspect import getmembers, isclass
from importlib.util import spec_from_file_location, module_from_spec
import openbb_terminal


def load_module(path: Path):
    """Loads a module from a given Path.
    Parameters:
    ----------
    path: Path
        The path to the module
    Returns:
    ----------
    The python module
    """
    spec = spec_from_file_location(path.stem, path)
    mod = module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod


def get_controller_path_list():
    module_path_generator = Path(openbb_terminal.__file__).parent.glob(
        "**/*controller.py"
    )
    module_path_list = list(module_path_generator)
    return module_path_list


try:
    import darts  # pylint: disable=W0611 # noqa: F401

    # If you just import darts this will pass during pip install, this creates
    # Failures later on, also importing utils ensures that darts is installed correctly
    from darts import utils  # pylint: disable=W0611 # noqa: F401

    FORECASTING = True
except ImportError:
    FORECASTING = False


def get_module_class_list(module):
    class_list = getmembers(module, isclass)
    class_name_list = [cls[0] for cls in class_list]

    return class_name_list


def build_controller_name(module):
    name_split = module.__name__.split("_")
    class_name = "".join([name.title() for name in name_split])
    return class_name


def get_controller_class_list():
    controller_path_list = get_controller_path_list()
    controller_class_list = []

    for controller_path in controller_path_list:
        if not FORECASTING and "forecast" in str(controller_path):
            continue

        module = load_module(path=controller_path)
        module_class_list = get_module_class_list(module=module)
        expected_class_name = build_controller_name(module=module)

        if expected_class_name in module_class_list:
            controller_class_list.append(getattr(module, expected_class_name))
        else:
            # I think python class should watch the module name
            # It's there a convention like this?
            print(controller_path)

    return controller_class_list


controller_class_list = get_controller_class_list()

print(controller_class_list)